In [1]:
import pandas as pd
import numpy as np
import os

In [13]:
# dataset directory
root_dir = r'D:\iHT\texture_recognition_pressure\dataset'

In [12]:
# dataframe with name of videos
training_data = pd.DataFrame()
videos = []
tags = os.listdir(root_dir)

for dir in os.listdir(root_dir):
    folder_path = os.path.join(root_dir, dir)

    for vid_files in os.listdir(folder_path):
        videos.append(vid_files)

training_data['videos'] = videos


## Dataset loading

In [14]:
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image

In [4]:
from PIL import Image
import imageio
import os

from torchvision import transforms

import torch
from torch.utils.data import Dataset



In [8]:
class GifDataset(Dataset):
    def __init__(self, root_dir, transform=None, fps=30):
        self.root_dir = root_dir
        self.transform = transform
        self.fps = fps
        self.gif_files = []
        self.labels = []
        self._load_gif_files()

    def _load_gif_files(self):
        for sample_dir in os.listdir(self.root_dir):
            sample_path = os.path.join(self.root_dir, sample_dir)
            if os.path.isdir(sample_path):
                for gif_file in os.listdir(sample_path):
                    if gif_file.endswith('.gif'):
                        self.gif_files.append(os.path.join(sample_path, gif_file))
                        self.labels.append(sample_dir)

    def _extract_frames(self, gif_file):
        gif = imageio.get_reader(gif_file)
        duration = gif.get_meta_data()['duration'] / 1000.0  
        total_frames = gif.count_frames()
        frame_interval = 1.0 / self.fps  

        frames = []

        for t in range(0, int(duration * self.fps)):
            frame_time = t * frame_interval
            frame_index = int(frame_time / (duration / total_frames))
            frame = gif.get_data(frame_index)
            frame_image = Image.fromarray(frame)
            if self.transform:
                frame_image = self.transform(frame_image)
            frames.append(frame_image)

        return frames

    def __len__(self):
        return len(self.gif_files)

    def __getitem__(self, idx):
        gif_file = self.gif_files[idx]
        frames = self._extract_frames(gif_file)
        label = self.labels[idx]
        return frames, label

# Example usage
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

root_dir = r'D:\iHT\texture_recognition_pressure\dataset'
dataset = GifDataset(root_dir, transform=transform, fps=30)

from torch.utils.data import DataLoader

batch_size = 4
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [ ]:
model.eval()

feature_vectors = []
labels = []

with torch.no_grad(), tqdm(total=len(dataset), desc=f'Extracting Features') as pbar:
    for frames, label in dataset:
        frames = frames.to(device)

        if hasattr(model, 'lstm'):
            model.lstm.reset_parameters()

        features_vector = model(frames.unsqueeze(0))

        feature_vectors.append(features_vector.squeeze().cpu().numpy())
        labels.append(label)
        pbar.update(1)

feature_vectors = np.array(feature_vectors)
labels = np.array(labels)